<a href="https://colab.research.google.com/github/TAUforPython/BioMedAI/blob/dev/example_fine_tune_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://habr.com/ru/articles/832984/

https://huggingface.co/datasets/ruslanmv/ai-medical-chatbot

In [1]:
%%capture

%pip install -U peft
%pip install -U trl
%pip install -U bitsandbytes

In [2]:
%pip install -U transformers # для использования модели
%pip install -U datasets # для подгрузки тестового датасетов
%pip install -U accelerate # вспомогательная библиотека
%pip install -U peft # QLoRA
%pip install -U trl # super-vised тренер
%pip install -U bitsandbytes # оптимизация весов модели
%pip install -U wandb # храним логи правильно

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 42.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.6/336.6 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.2.1
    Uninstalling accelerate-1.2.1:
      Successfully uninstalled accelerate-1.2.1


In [3]:
# Импорт библиотек
import os, torch, wandb

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)

from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format
from dataclasses import dataclass

from huggingface_hub import login

In [4]:
import os
from google.colab import userdata
# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.
os.environ["HF_token"] = userdata.get("HF_token")

In [5]:
from huggingface_hub import login

login(os.environ["HF_token"])

In [25]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.

os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

In [34]:
# Настройка Weights & Bias для логирования при обучении


user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("huggingface_token")

login(token = hf_token)

wb_token = user_secrets.get_secret("wandb_api_key")

wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tune Llama 3.1 8B on Medical Dataset',
    job_type="training",
    anonymous="allow"
)

CredentialError: A JWT Token is required to call Kaggle, but none found in environment variable KAGGLE_USER_SECRETS_TOKEN

In [9]:
@dataclass
class Config:
#     model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
#     model_name = "AnatoliiPotapov/T-lite-instruct-0.1"
    model_name = "Qwen/Qwen2.5-0.5B-Instruct"
    dataset_name = "ruslanmv/ai-medical-chatbot"
    new_model = "llama-3.1-8b-chat-doctor"
    torch_dtype = torch.float16
    attn_implementation = "eager"
cfg = Config()

In [11]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=cfg.torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    cfg.model_name,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=cfg.attn_implementation
)

CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend


RuntimeError: CUDA is required but not available for bitsandbytes. Please consider installing the multi-platform enabled version of bitsandbytes, which is currently a work in progress. Please check currently supported platforms and installation instructions at https://huggingface.co/docs/bitsandbytes/main/en/installation#multi-backend

In [7]:
# Вспомогательный класс для сохранения атрибутов

@dataclass
class Config:
    #model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"
    model_name = "Qwen/Qwen2.5-0.5B-Instruct"
    dataset_name = "ruslanmv/ai-medical-chatbot"
    new_model = "llama-3.1-8b-chat-doctor"
    torch_dtype = torch.float16
    attn_implementation = "eager"


# напишем вспомогательный класс для хранений атрибутов
cfg = Config()

# cfg можно будет потом использовать вот так удобно
tokenizer = AutoTokenizer.from_pretrained(cfg.model_name)

model, tokenizer = setup_chat_format(model, tokenizer)
tokenizer.padding_side = 'right'
tokenizer.padding_token = '<|pad|>'

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

NameError: name 'model' is not defined

Теперь создадим все классы для обучения модели. Так как у нас нет времени для полного обучения LLM, то мы воспользуемся эффективным по памяти адаптером Q-LoRA, которая позволяет поместить модель на видеокарту 16GB.

In [ ]:
# Настройка адаптера (QLoRa) для быстрого обучения
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)

# LoRA config / адаптер
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)

In [ ]:
# Подготовка датасета
def format_chat_template(row):
    row_json = [{"role": "user", "content": row["Patient"]},
               {"role": "assistant", "content": row["Doctor"]}]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row


# Библиотека Datasets 🤗 очень приятна в использовании,
# когда используете LLM. Так что берите на вооружение, чемпионы!
dataset = load_dataset(cfg.dataset_name, split="all")

dataset = dataset.map(
	format_chat_template,
	num_proc=4,
)


# Для учебных целей урежем исходный датасет,
# чтобы сравнить результаты - читаем в самом конце
dataset_sh = dataset.shuffle(seed=2024).select(range(10_000))

In [ ]:
training_arguments = TrainingArguments(
    output_dir=cfg.new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    eval_strategy="steps",
    eval_steps=0.2,
    logging_steps=100,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="wandb"
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_sh["train"],
    eval_dataset=dataset_sh["test"],
    peft_config=peft_config, # сам адаптер, который создали ранее
    max_seq_length=512,
    dataset_text_field="text",
    tokenizer=tokenizer, # был импортирован
    args=training_arguments,
    packing= False,
)

trainer.train() # Запускаем обучение